## A demonstration of PANDAS data frames used to investigate CAO points

Author: Jon Ishaque
Commenced: 29th September 2021
GMIT SID: G00398244

This notebook extracts CAO points from the CAO website for 2019, 2020 and 2021. It loads data into pandas dataframes and uses pandas and python to compare points from different years.

 Documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

In [49]:
# package for making http requests

import requests as rq
# Dates and time package
import datetime as dt

#dataframes
import pandas as pd

#import regex package for searching strings
import re

#import csv, deals with commas when writing to file
import csv

## 2021 points
#http://www.cao.ie/index.php?page=points&p=2021
The 2021 CAO points are presented in a web page. 

The page header from the server should decode as per: *Content-Type: text/html; charset=iso-8859-1*
However, one line uses \x96 which isn't defined in iso-8859-1. Therefore we use the similar decoding standard cp1252, which is very similar but includes #x96. The character in question was had an Irish foda on a level 8 course

In [50]:
#


Create a string var,*now*. this is use in file names of back up copies of CAO points.

In [51]:
# Get the current date and time.

now = dt.datetime.now()

# Format as a string.
#global as used in functions
global nowstr
nowstr = now.strftime('%Y%m%d_%H%M%S')



###### Compile the regular expression so it is not compiled at each interation of the loop reading the webpage

###### Explanation of the regualar expression [4][5]:
('[A-Z]{2}[0-9]{3} (.*)([0-9]{3}))</span>

[A-Z]{2}        Any two upper case aphanumberic

[0-9]{3}        Any three digits 0-9

'  '            Two spaces

(.*)([0-9]{3})   Any amount of text before 3 numeric characters


    </font>

In [52]:
#set reg ex
re_courses = re.compile('[A-Z]{2}[0-9]{3} (.*)') #[4]




In [53]:
#Function to get HEI name from course code. using a switcher dict as
#oppose to messy if/else block
#https://www.upgrad.com/blog/how-to-implement-switch-case-functions-in-python/
def getHEI(cc):
    switcher = {'AC' : 'American College',
    'AD' : 'National College of Art and Design',
    'AL' : 'Athlone Institute of Technology',
    'AS' : 'St. Angela`s College',
    'CI' : 'Irish College of Humanities & Applied Sciences',
    'BY' : 'IBAT College Dublin',
    'CK' : 'University College Cork (NUI)',
    'CM' : 'Marino Institute of Education',
    'CR' : 'Cork Institute of Technology',
    'CT' : 'CCT College Dublin',
    'CW' : 'Institute of Technology Carlow',
    'DB' : 'Dublin Business School',
    'DC' : 'Dublin City University',
    'DK' : 'Dundalk Institute of Technology',
    'DL' : 'Dun Laoghaire Institute of Art Design and Technology',
    'DN' : 'University College Dublin (NUI)',
    'DS' : 'Dorset College',
    'GA' : 'Galway-Mayo Institute of Technology',
    'GB' : 'Galway Business School',
    'GC' : 'Griffith College',
    'GY' : 'National University of Ireland Galway',
    'ID' : 'ICD Business School',
    'LC' : 'Limerick Institute of Technology',
    'LM' : 'University of Limerick',
    'LY' : 'Letterkenny Institute of Technology',
    'MH' : 'Maynooth University',
    'MI' : 'Mary Immaculate College',
    'MU' : 'Pontifical University St Patricks College',
    'NC' : 'National College of Ireland (NCI)',
    'NM' : 'St Nicholas Montessori College Ireland',
    'PC' : 'Carlow College St. Patricks',
    'RC' : 'RCSI University of Medicine & Health Sciences',
    'SG' : 'Institute of Technology Sligo',
    'TL' : 'Institute of Technology Tralee',
    'TR' : 'Trinity College Dublin',
    'TU' : 'Technological University Dublin',
    'WD' : 'Waterford Institute of Technology'
    }

    cc = cc[:2]
    return  switcher.get(cc)

#print(HEI('WD123123'))


In [54]:
#helper
def points_to_arr(s):
    AQA=''
    portfolio =''
    points=''
    random = ''
    #check 1st char for #
    #print(s)
    if s[0]=='#':
        portfolio='#'# add to var
    random = ''
    #check final char for  *
    if s[-1] == '*':
        random ='*'
    points=''
    
    if s.find("AQA") ==-1: #not AQA
        #strip ~ and * from start and end of s
        for i in s:
            if i.isdigit():
                #concat points string
                points = points + i
    else:
        AQA ="AQA" #return AQA as separate val as it will be separate column
        #return
    return [points, portfolio, random,AQA]

The following block of code iterates through each line of the csv file 

This part of the note part of the note book will load the web page content. A loop will read each line of web page and determine if it's content is relevant and write content to a csv file.

In [55]:
#MAKE THIS BLOCK A FUNCTION AS REPEATING IT FOR L8 and L6/7
#get a save the csv names to paths
global csv_files
csv_files = []
def createCSV(path):
    print (path)
    #Get the both level 8 and 6/7 web pages
    #getheaders and determine contenttype [3]

    
    
#respL8.headers['content-type']
        
   
    #resp.text
    #loop through response text lines
    #get level
    print (path)
    
    if path.find('L8') >= 0 :
        #print (path)
        level = '8'
        
        resp = rq.get('http://www2.cao.ie/points/l8.php', 
                      headers={"content-type":"text"})
        
    elif path.find('L67') >= 0 :
        #print (path)
        level ='6/7'
        
        resp = rq.get('http://www2.cao.ie/points/l76.php', 
                      headers={"content-type":"text"})
    else:
        level = ''
        
    
    original_encoding = resp.encoding
    # Change to cp1252, which handles accented characters
    resp.encoding = 'cp1252'
     # Create a file path for the original data. 2021
    pathhtml = path + nowstr + '.html'
    # Save the original html file.
    with open(pathhtml, 'w') as f:
        f.write(resp.text)

    #set var to count lines for cross check with webpage
    no_lines = 0
    path = path+'.csv'
    #add csv name to array
    csv_files.append(path)
    with open(path, 'w') as f:
        #write csv header
        linesplit = ['Course code','Course title','Round 1',
                     'Portfolios_other1','Random 1','AQA1','Round 2',
                     'Portfolios_other2','Random 2','AQA2',
                     'HEI','Level','Year']
        f.write(','.join(linesplit) + '\n')
        for line in resp.iter_lines():
            #

            #problem with bytes
            #so convert str to bytes
            #print (line)
            dline = line.decode('cp1252')
            #check if line mathces reg exp pattern. If so, do something.
            if re_courses.fullmatch(dline):
                no_lines +=1
                #get first five chars - course code
                course_code = dline[:5]
                #course title
                course_title = dline[7:57]
                #r1 points
                round_1 = dline[60:65].rstrip() # get five chars, remove white space
                #if round 1 not blank call fn points_to_arr
                if len(round_1) > 0:
                    round_1= points_to_arr(round_1)
                    #assign vals from returned array
                    pts1 = round_1[0]
                    plo1 = round_1[1]
                    rnd1 = round_1[2]
                    AQA1 = round_1[3]
                else: 
                    pts1 = ''
                    plo1 = ''
                    rnd1 = ''
                    AQA1 = ''
                #r2 points
                round_2 = dline[67:].rstrip() # get four chars, remove white space
                #if round 2 not blank call fn points_to_arr
                if len(round_2) > 0:
                    round_2= points_to_arr(round_2)
                    #assign vals from returned array
                    pts2 = round_2[0]
                    plo2 = round_2[1]
                    rnd2 = round_2[2]
                    AQA2 = round_2[3]
                else: 
                    pts2 = ''
                    plo2 = ''
                    rnd2 = ''
                    AQA2 = ''
                #print (course_code)
                #get the instituion name
                HEI =getHEI(course_code)
                #print (HEI)
                # create an array of the fields for the csv line
                linesplit = [course_code,course_title,pts1,plo1,rnd1,AQA2,pts2,plo2,rnd2,AQA2,HEI,level,'2021']
                #print (linesplit)
                #debug
                #print(f"'{course_code} {dline} r1: {round_1} r2: {round_2}'")
               # print((','.join(linesplit) + '\n'))
                # Rejoin the array values with commas in between. ie.comma separated
                f.write(','.join(linesplit) + '\n')
    print (f"number of lines is", {no_lines})
    path=''
#check this number is correct


In [56]:
# The file path for the csv file.
pathL8 = 'data/cao2021_L8_' + nowstr 
pathL67 ='data/cao2021_L67_' + nowstr 

createCSV(pathL8)
createCSV(pathL67)

data/cao2021_L8_20211116_120515
data/cao2021_L8_20211116_120515
number of lines is {949}
data/cao2021_L67_20211116_120515
data/cao2021_L67_20211116_120515
number of lines is {416}


#### NB: 949 L8 courses on CAO website verified on 10th November 2021
#### 416 L6/7 courses on CAO website verified on 15th November 2021

Join L8 & L6/7 courses into one dataframe

In [57]:
# loop over the list of csv files
#https://stackoverflow.com/questions/16597265/appending-to-an-empty-dataframe-in-pandas

print(csv_files)
df = pd.DataFrame()
for f in csv_files:
      
    # read the csv file
    print(f)
    df_temp = pd.read_csv(f)
    
    
    df = df.append(df_temp, ignore_index = True)
print(df)
    


['data/cao2021_L8_20211116_120515.csv', 'data/cao2021_L67_20211116_120515.csv']
data/cao2021_L8_20211116_120515.csv
data/cao2021_L67_20211116_120515.csv
     Course code                                       Course title  Round 1  \
0          AL801  Software Design for Virtual Reality and Gaming...    300.0   
1          AL802  Software Design in Artificial Intelligence for...    313.0   
2          AL803  Software Design for Mobile Apps and Connected ...    350.0   
3          AL805  Computer Engineering for Network Infrastructur...    321.0   
4          AL810  Quantity Surveying                            ...    328.0   
...          ...                                                ...      ...   
1360       WD188  Applied Health Care                           ...    220.0   
1361       WD205  Molecular Biology with Biopharmaceutical Scien...      NaN   
1362       WD206  Electronic Engineering                        ...    180.0   
1363       WD207  Mechanical Engineering       

In [58]:
#delete duplicate rows

*** 

## 2020 CAO points
###### http://www.cao.ie/index.php?page=points&p=2020 CAO points in 2020 include level 6,7 & 8

In [59]:
#use urlib to retrieve url as file
import urllib.request as urlrq

In [60]:
# Create a file path for the original data.For backup
path = 'data/cao2020_' + nowstr + '.xlsx'

#download to path
urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx',\
                  path)

('data/cao2020_20211116_120515.xlsx',
 <http.client.HTTPMessage at 0x21823333d90>)

In [61]:
###### Read the Excel file into a pandas dataframe 

In [62]:
#download and parse the excel spreadsheet
#skip first 10 rows
df=pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx',\
                 skiprows=10)


In [63]:
#df.iloc[123]

#check final row
#delete unwanted columns
df = df.drop(['CATEGORY (i.e.ISCED description)','avp','v','Column1',\
              'Column2','Column3','Column4','Column5','Column6',\
              'Column7','Column8'], 1)
df = df.rename(columns={'COURSE TITLE': 'Course title',\
                        'COURSE CODE2': 'Course code',\
                        'R1 POINTS':'Round 1','R2 POINTS':'Round 2',\
                        'R1 Random *':'Random 1',\
                        'R2 Random*':'Random 2'})

df['Year'] =2020
df['Portfolios_other1'] =''
df['Portfolios_other2'] =''
df['AQA1'] =''
df['AQA2'] =''
#https://towardsdatascience.com/check-for-a-substring-in-a-pandas-dataframe-column-4b949f64852

#Pulling out AQA, # and  placing into own column.
df.loc[df['Round 1'].str.contains('#',na=False) ,\
       'Portfolios_other1'] = '#' 
df.loc[df['Round 1'].str.contains('#',na=False) , 'Round 1'] = '' 
df.loc[df['Round 2'].str.contains('#',na=False) ,\
       'Portfolios_other2'] = '#' 
df.loc[df['Round 2'].str.contains('#',na=False) , 'Round 2'] = '' 
df.loc[df['Round 1'].str.contains('AQA',na=False) , 'AQA1'] = 'AQA' 
df.loc[df['Round 2'].str.contains('AQA',na=False) , 'AQA2'] = '' 
display (df.loc[df['Course code']=='CR210']) # check we are picking up commas in csv fields          

,Course title,Course code,Round 1,Random 1,Round 2,Random 2,EOS,EOS Random *,EOS Mid-point,LEVEL,HEI,Test/Interview #,Year,Portfolios_other1,Portfolios_other2,AQA1,AQA2
195,"Contemporary Applied Art (Ceramics, Glass, Tex...",CR210,,NaN,NaN,NaN,#+matric,NaN,#+matric,8,Cork Institute of Technology,#,2020,#,,,


***

## 2019 CAO
#http://www2.cao.ie/points/lvl8_19.pdf

In [64]:
import camelot #use camelot package to extract tables from pdf files [7]

In [65]:
#create a path to back up the file as a csv
path = 'data/cao2019_' + nowstr + '.csv'


In [66]:
tables = camelot.read_pdf('http://www2.cao.ie/points/lvl8_19.pdf',\
                          pages='1-end',flavor='stream')
#read all pages [8]

tables
tbl_cnt = len(tables)

#export all tables - not what we really want
#tables.export(path, f='csv', compress=False) # json, excel, html, markdown, sqlite
#tables[0]

tables[0].parsing_report
{
    'accuracy': 99.02,
    'whitespace': 12.24,
    'order': 1,
    'page': 10
}

{'accuracy': 99.02, 'whitespace': 12.24, 'order': 1, 'page': 10}

print parsing report of first table

In [67]:
tbl_cnt

18

In [68]:
# The file path for the csv file.
path = 'data/cao2019_csv_' + nowstr + '.csv'

In [69]:
i = 1 # exclude first header table 
#interate through the list of tables [9] 
data2019 = [] # empty list of tables

for t in tables:    
    if i > 0: #exclude 1st table
        #write the table as a dataframe to listdata2019
        
        data2019.append(t.df) 
    i +=1 
    
#combine all the dataframes in the list into one dataframe
dfcombined = pd.concat(data2019)

#add column headers
dfcombined.columns = ['Course Code', 'COURSE', 'EOS', 'Mid']

#write to csv to store as back up.
dfcombined.to_csv(path)


Filter df so only rows with course codes remain. [10]

In [70]:
def regex_filter(val): 
    regex= '[A-Z]{2}[0-9]{3}'
    if val:
        mo = re.search(regex,val)
        if mo:
            return True
        else:
            return False
    else:
        return False

df_filtered = dfcombined[dfcombined['Course Code'].apply(regex_filter)]





###### reset index to remove indexes from appended dataframes.
reset because reindex will notwork with duplicate values indexes [11]


In [71]:
df = df_filtered.reset_index(drop=True)

In [72]:
df

,Course Code,COURSE,EOS,Mid
0,AL801,Software Design with Virtual Reality and Gaming,304,328
1,AL802,Software Design with Cloud Computing,301,306
2,AL803,Software Design with Mobile Apps and Connected...,309,337
3,AL805,Network Management and Cloud Infrastructure,329,442
4,AL810,Quantity Surveying,307,349
...,...,...,...,...
925,WD200,Arts (options),221,296
926,WD210,Software Systems Development,271,329
927,WD211,Creative Computing,275,322
928,WD212,Recreation and Sport Management,274,311


In [73]:
display (df.loc[df['Course Code']=='AL861'])
#deal with *, ~ etc

,Course Code,COURSE,EOS,Mid
23,AL861,Animation and Illustration (portfolio),#615,899


---
## References:
[1]

[2]

[3] https://docs.python-requests.org/en/latest/index.html

[4] https://docs.python.org/3/library/re.html

[5] https://docs.python.org/3/library/re.html?highlight=re%20match#re.match

[6] https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html?highlight=read_excel#pandas.read_excel

[7] https://camelot-py.readthedocs.io/en/master/

[8] https://github.com/atlanhq/camelot/issues/278

[9] https://stackoverflow.com/questions/55052989/how-to-iterate-through-a-list-of-data-frames-and-drop-all-data-if-a-specific-str

[10] https://stackoverflow.com/questions/15325182/how-to-filter-rows-in-pandas-by-regex/48884429

[11] https://stackoverflow.com/questions/68261366/right-way-to-reindex-a-dataframe


## End